In [12]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

In [13]:
PM10 = pd.read_csv('./data/PM10-2023.csv', sep=';')
NO2 = pd.read_csv('./data/NO2-2023.csv', sep=';')
traffic = pd.DataFrame()
for file in os.listdir('./data/natezenie_ruchu_2023'):
    data = pd.read_csv('./data/natezenie_ruchu_2023/' + file, sep=';')
    traffic = pd.concat([traffic, data])

In [14]:
PM10 = PM10.drop_duplicates()
traffic = traffic.drop_duplicates()
NO2 = NO2.drop_duplicates()

In [15]:
traffic.columns = ['Traffic_station_id', 'date', 'hour', 'quality', 'num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed', 'num_trucks', 'avg_truck_speed']
df = pd.read_excel('./data/Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
# a = []
# for i, item in enumerate(df['DET_ID15']):
#     if item in traffic['station_id'].unique():
#         a.append([df['x'][i], df['y'][i]])

In [16]:
traffic_id = [100101010031590, 100101010031287]
traffic['date'] = pd.to_datetime(traffic['date'], format="%d.%m.%Y")
traffic['hour'] = pd.to_timedelta(traffic['hour'], unit='h')
traffic['datetime'] = traffic['date'] + traffic['hour']
traffic = traffic.drop(columns=['date', 'hour'])
traffic = traffic[traffic['Traffic_station_id'].isin(traffic_id)]

In [17]:
PM10['Date'] = pd.to_datetime(PM10['Date'], format="%Y-%m-%d")
PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
PM10['Time'][PM10['Time'] == '24:00'] = '00:00'
PM10['Time'] = pd.to_timedelta(PM10['Time'].str.strip("'") + ':00')
PM10['datetime'] = PM10['Date'] + PM10['Time']
PM10 = PM10.drop(columns=['Date', 'Time', "State / Measuring network", "Station setting","Station type","Pollutant","Time scope", "Unit", "Station name"])

C:\Users\wpietron\AppData\Local\Temp\ipykernel_14604\72315281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
C:\Users\wpietron\AppData\Local\Temp\ipykernel_14604\72315281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Time'][PM10['Time'] == '24:00'] = '00:00'


In [18]:
print(len(traffic))
print(len(PM10))
PM10.to_csv("pm.csv", index=False)
traffic.to_csv("traffic.csv", index=False)

13432
17518


In [19]:
#date merge
merged_df = pd.merge(traffic, PM10, left_on='datetime', right_on='datetime', how='inner')

#split by station code

# print(len(merged_df))
# merged_df.drop_duplicates(subset=['datetime', 'station_id'], inplace=True)
# print(len(merged_df))

merged_df.to_csv("merged.csv", index=False)
# #count nan
# print(merged_df.isna().sum())


In [25]:
# ined_df_4 = merged_df.pivot_table(index=['datetime'], columns='CellName', values=['load', 'ESMode1', 'ESMode2', 'ESMode3',
#        'ESMode4', 'ESMode5', 'ESMode6', 'Frequency', 'Bandwidth', 'TXpower'], aggfunc='sum', fill_value=999)
pivot_df = merged_df.pivot_table(index=['datetime'],
                          columns=['Station code', 'Traffic_station_id'],
                          values=['num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed',
                                  'num_trucks', 'avg_truck_speed', 'Measure value','quality'],
                          aggfunc='first')

# Flatten the multi-index columns
pivot_df.columns = ['_'.join(str(col)).strip() for col in pivot_df.columns.values]

# Reset index to make 'station_id', 'quality', and 'datetime' as regular columns
pivot_df.reset_index(inplace=True)

# Display the combined DataFrame
print(pivot_df)
pivot_df.to_csv("pivot.csv", index=False)

                datetime  \
0    2023-01-05 12:00:00   
1    2023-01-05 13:00:00   
2    2023-01-05 14:00:00   
3    2023-01-05 15:00:00   
4    2023-01-05 16:00:00   
...                  ...   
6714 2023-12-08 00:00:00   
6715 2023-12-08 01:00:00   
6716 2023-12-08 02:00:00   
6717 2023-12-08 03:00:00   
6718 2023-12-08 04:00:00   

     (_'_M_e_a_s_u_r_e_ _v_a_l_u_e_'_,_ _'_D_E_B_E_0_6_8_'_,_ _1_0_0_1_0_1_0_1_0_0_3_1_2_8_7_)  \
0                                                     6                                          
1                                                     6                                          
2                                                     6                                          
3                                                     6                                          
4                                                     6                                          
...                                                 ...                    